In [2]:
import pandas as pd
import numpy as np
import warnings
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.2f}'.format

# Trabalhando os Dados Iniciais

Critérios de corte Inicial:
Importações menores que 1000 litros no total de 15 anos.
Países que só fizeram 1 ou 2 compras no período de 15 anos e não foram nos anos de 2021 ou 2022 (Compradores eventuais)
Corte da Rússia que causa uma distorção nos dados por conta de um acordo comercial entre Brasil e Rússia que fez o ano de 2009 ter um volume fora da realidade. Sem contar que a indefinição da guerra com a Ukrânia nao inspira confiança em investir em uma parceria neste momento.

In [3]:
df = pd.read_excel('./ExportacaoVinhos.xlsx')
df_inicial = df[df.columns[0:31]]
df_inicial.head()

,Pais,2008_KG,2008_USD,2009_KG,2009_USD,2010_KG,2010_USD,2011_KG,2011_USD,2012_KG,2012_USD,2013_KG,2013_USD,2014_KG,2014_USD,2015_KG,2015_USD,2016_KG,2016_USD,2017_KG,2017_USD,2018_KG,2018_USD,2019_KG,2019_USD,2020_KG,2020_USD,2021_KG,2021_USD,2022_KG,2022_USD
0,Paraguai,2191901,1374088,486927,392087,510989,449197,240168,276281,354824,428279,481564,680828,521847,908028,495428,741370,985739,1655417,2393468,4274650,3234168,5494321,2419537,3826587,3299013,3869243,6522527,7192362,5076670,7156293
1,Estados Unidos,443895,804607,372319,660066,228968,478630,306787,1030254,146585,303986,245368,786556,222267,494216,195896,524109,258072,687411,132688,1523699,169109,512519,209765,616274,300178,610793,111085,203554,220373,447893
2,China,8689,25926,1553416,482400,795,2358,54156,334867,87905,642177,40929,279956,64040,455340,47609,222866,134106,499622,67594,266086,30835,126336,129852,376828,122253,363000,61884,264116,105395,404647
3,Espanha,2942,6834,2181,4050,0,0,5206,24618,0,0,1972980,3748940,0,0,0,0,0,0,0,0,6123,22631,3540,1353,28,126,0,0,0,0
4,Haiti,20,20,4500,5863,2700,3750,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79500,144425,81873,129803,399128,471152,670379,831181,553503,741014


In [4]:
df_inicial.shape

(71, 31)

In [5]:
df_melted = pd.melt(df_inicial, id_vars=['Pais'], var_name='Ano e Valor', value_name='Quantidade')
df_melted[['Ano', 'Valor']] = df_melted['Ano e Valor'].str.split('_', expand=True)
df_exportacao = df_melted.pivot(index=['Pais', 'Ano'], columns='Valor', values='Quantidade').reset_index()
df_exportacao.columns.name = None 
df_exportacao = df_exportacao.rename(columns={'KG': 'Litros', 'Pais' : 'Destino'})
df_exportacao['Origem'] = 'Brasil'
df_exportacao['Ano'] = pd.to_datetime(df_exportacao['Ano'], format='%Y')
df_exportacao['Litros'] = df_exportacao['Litros'].astype(float)
df_exportacao['USD'] = df_exportacao['USD'].astype(float)
df_exportacao.head()

,Destino,Ano,Litros,USD,Origem
0,Alemanha,2008-01-01,"265,742.00","429,970.00",Brasil
1,Alemanha,2009-01-01,"225,086.00","393,482.00",Brasil
2,Alemanha,2010-01-01,"27,715.00","138,666.00",Brasil
3,Alemanha,2011-01-01,"36,070.00","144,150.00",Brasil
4,Alemanha,2012-01-01,"8,189.00","56,342.00",Brasil


In [6]:
df_exportacao.shape

(1065, 5)

In [7]:
df_exportacao.describe()

,Ano,Litros,USD
count,1065,"1,065.00","1,065.00"
mean,2015-01-01 03:12:00,"45,938.76","81,684.62"
min,2008-01-01 00:00:00,0.00,0.00
25%,2011-01-01 00:00:00,0.00,0.00
50%,2015-01-01 00:00:00,468.00,"1,549.00"
75%,2019-01-01 00:00:00,"7,179.00","23,780.00"
max,2022-01-01 00:00:00,"6,522,527.00","7,192,362.00"
std,NaN,"330,010.49","449,592.23"


In [8]:
df_exportacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Destino  1065 non-null   object        
 1   Ano      1065 non-null   datetime64[ns]
 2   Litros   1065 non-null   float64       
 3   USD      1065 non-null   float64       
 4   Origem   1065 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 41.7+ KB


In [9]:
df_exportacao['Destino'].nunique()

71

#### Anotação: Nos últimos 15 anos nós exportamos vinho para 71 Países.

# Gerando a Tabela Solicitada Pelo Head de Dados

In [10]:
df_total_por_pais = df_exportacao.groupby(['Origem','Destino']).sum(['Litros','USD']).sort_values('Litros', ascending=False)
df_total_por_pais.reset_index(inplace=True)
df_total_por_pais.head()

,Origem,Destino,Litros,USD
0,Brasil,Paraguai,"29,214,770.00","38,719,031.00"
1,Brasil,Estados Unidos,"3,563,355.00","9,684,567.00"
2,Brasil,China,"2,509,458.00","4,746,525.00"
3,Brasil,Espanha,"1,993,000.00","3,808,552.00"
4,Brasil,Haiti,"1,791,603.00","2,327,208.00"


In [11]:
df_total_por_pais.to_excel('./ExportacaoVinhosTotais.xlsx', index=False)

# Trabalhando com os dados totais dos últimos 15 anos

In [12]:
df_total_por_pais['USD_por_Litro'] = df_total_por_pais['USD'] / df_total_por_pais['Litros']
df_total_por_pais.head(10)

,Origem,Destino,Litros,USD,USD_por_Litro
0,Brasil,Paraguai,"29,214,770.00","38,719,031.00",1.33
1,Brasil,Estados Unidos,"3,563,355.00","9,684,567.00",2.72
2,Brasil,China,"2,509,458.00","4,746,525.00",1.89
3,Brasil,Espanha,"1,993,000.00","3,808,552.00",1.91
4,Brasil,Haiti,"1,791,603.00","2,327,208.00",1.30
5,Brasil,Reino Unido,"1,239,551.00","4,711,464.00",3.80
6,Brasil,Paises Baixos,"1,236,154.00","3,791,611.00",3.07
7,Brasil,Japão,"1,181,692.00","2,377,716.00",2.01
8,Brasil,Alemanha,"909,051.00","2,546,394.00",2.80
9,Brasil,Uruguai,"792,595.00","1,219,878.00",1.54


### Optei por trabalhar com os dados dos top 10 em volume de exportação que representam 90,82% dos nossos dados.

In [13]:
top10 = df_total_por_pais.sort_values('Litros', ascending=False)['Destino'].head(10).to_list()
top10

['Paraguai',
 'Estados Unidos',
 'China',
 'Espanha',
 'Haiti',
 'Reino Unido',
 'Paises Baixos',
 'Japão',
 'Alemanha',
 'Uruguai']

In [14]:
df_total_top10 = df_total_por_pais[df_total_por_pais['Destino'].isin(top10)]
df_total_top10

,Origem,Destino,Litros,USD,USD_por_Litro
0,Brasil,Paraguai,"29,214,770.00","38,719,031.00",1.33
1,Brasil,Estados Unidos,"3,563,355.00","9,684,567.00",2.72
2,Brasil,China,"2,509,458.00","4,746,525.00",1.89
3,Brasil,Espanha,"1,993,000.00","3,808,552.00",1.91
4,Brasil,Haiti,"1,791,603.00","2,327,208.00",1.30
5,Brasil,Reino Unido,"1,239,551.00","4,711,464.00",3.80
6,Brasil,Paises Baixos,"1,236,154.00","3,791,611.00",3.07
7,Brasil,Japão,"1,181,692.00","2,377,716.00",2.01
8,Brasil,Alemanha,"909,051.00","2,546,394.00",2.80
9,Brasil,Uruguai,"792,595.00","1,219,878.00",1.54


In [15]:
df_exportacao_top10 = df_exportacao[df_exportacao['Destino'].isin(top10)]
df_exportacao_top10.head()

,Destino,Ano,Litros,USD,Origem
0,Alemanha,2008-01-01,"265,742.00","429,970.00",Brasil
1,Alemanha,2009-01-01,"225,086.00","393,482.00",Brasil
2,Alemanha,2010-01-01,"27,715.00","138,666.00",Brasil
3,Alemanha,2011-01-01,"36,070.00","144,150.00",Brasil
4,Alemanha,2012-01-01,"8,189.00","56,342.00",Brasil


In [16]:

fig = px.bar(df_total_top10, x='Destino', y=['Litros'], title='Distribuição em Volume por Destino')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Países',
    xaxis_title_standoff=55,
    yaxis_title='Milhões de Litros',
    yaxis_type="log",
    yaxis_showticklabels=False,
    showlegend=False, 
    template="plotly_dark",
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)

fig.show()

In [17]:
fig = px.bar(df_total_top10, x='Destino', y=['USD'], title='Distribuição em Dólares por Destino')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Países',
    xaxis_title_standoff=55,
    yaxis_title='Milhões de Dólares',
    yaxis_type="log",
    yaxis_showticklabels=False,
    showlegend=False, 
    template="plotly_dark",
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='US$ %{y:.2s}', textposition='outside', textfont_size=10)

fig.show()

In [18]:
fig = px.bar(df_total_top10.sort_values(by='USD_por_Litro',ascending=False), x='Destino', y=['USD_por_Litro'], title='Valor em Dolar por Litro no Destino')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Países',
    xaxis_title_standoff=55,
    yaxis_title='Dólares',
    yaxis_type="log",
    yaxis_showticklabels=False,
    showlegend=False, 
    template="plotly_dark",
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='US$ %{y:.2s}', textposition='outside', textfont_size=10)

fig.show()

In [19]:
df_producao_exportacao = pd.read_excel('./ProducaoExportacao.xlsx')
df_producao_exportacao['Ano'] = pd.to_datetime(df_producao_exportacao['Ano'], format='%Y')
df_producao_exportacao['Produzido'] = df_producao_exportacao['Produzido'].astype(float)
df_producao_exportacao['Exportado'] = df_producao_exportacao['Exportado'].astype(float)
df_producao_exportacao['PTAX'] = df_producao_exportacao['PTAX'].astype(float)
df_producao_exportacao.head()

,Ano,Produzido,Exportado,PTAX
0,2008-01-01,"334,841,312.00","10,346,323.00",2.34
1,2009-01-01,"245,318,774.00","25,514,198.00",1.74
2,2010-01-01,"220,073,693.00","1,280,574.00",1.67
3,2011-01-01,"305,439,220.00","1,214,834.00",1.88
4,2012-01-01,"257,977,767.00","5,775,376.00",2.04


In [20]:
fig = px.bar(df_producao_exportacao.sort_values(by='Ano',ascending=False), x='Ano', y=['Produzido','Exportado'], title='Produzido vs Exportado', barmode='group')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Anos',
    xaxis_title_standoff=55,
    yaxis_title='Milhões de Litros',
    yaxis_type="log",
    yaxis_showticklabels=False,
    template="plotly_dark",
    xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)



fig.show()

In [21]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Produzido'], name='Produzido'))
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)
fig.add_trace(go.Scatter(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Produzido'], mode='lines', name='Tendência', line=dict(color='red')))

fig.update_layout(title='Variação da Produção Anual com Linha de Tendência',
                  barmode='group',
                  title_font=dict(size=24), 
                  xaxis_title='Anos',
                  xaxis_title_standoff=55,
                  yaxis_type="log",
                  yaxis_showticklabels=False,
                  showlegend=False, 
                  yaxis_title='Produção em Milhões de Litros',
                  template="plotly_dark",
                  xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
                  annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

fig.show()

#### Observação: Em 2016, a safra de uvas teve uma das maiores quebras registradas, com uma redução de 57% em relação ao ano anterior, equivalente, aproximadamente, a 300 milhões de quilos de uvas no território gaúcho. A principal causa da quebra histórica no ano 2016 foi uma série de acontecimentos climáticos que prejudicaram o desenvolvimento das uvas ao longo do ano, como geadas e excesso de chuvas.

In [22]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Exportado'], name='Exportado'))
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)
fig.add_trace(go.Scatter(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Exportado'], mode='lines', name='Tendência', line=dict(color='red')))
fig.update_layout(title='Variação da Exportação Anual com Linha de Tendência',
                  barmode='group',
                  title_font=dict(size=24), 
                  xaxis_title='Anos',
                  xaxis_title_standoff=55,
                  yaxis_type="log",
                  yaxis_showticklabels=False,
                  showlegend=False, 
                  yaxis_title='Exportação em Milhões de Litros',
                  template="plotly_dark",
                  xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
                  annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

fig.show()

In [23]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['PTAX'], name='PTAX'))
fig.update_traces(texttemplate='%{y:.4s}', textposition='outside', textfont_size=10)
fig.add_trace(go.Scatter(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['PTAX'], mode='lines', name='Tendência', line=dict(color='red')))
fig.update_layout(title='Variação da PTAX Anual com Linha de Tendência',
                  yaxis_title='PTAX',
                  barmode='group',
                  title_font=dict(size=24), 
                  xaxis_title='Anos',
                  yaxis_type="log",
                  xaxis_title_standoff=55,
                  yaxis_showticklabels=False,
                  showlegend=False,
                  template="plotly_dark",
                  xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
                  annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

fig.show()

In [24]:
total_produzido = df_producao_exportacao['Produzido'].sum()
total_exportado = df_producao_exportacao['Exportado'].sum()
dolar_medio = df_producao_exportacao['PTAX'].mean()
percentual_exportado= (total_exportado/total_produzido) * 100

print(f"Total Produzido: {total_produzido}")
print(f"Total Exportado: {total_exportado}")
print(f"Percentual Exportado: {percentual_exportado}")
print(f"Dolar Médio: {dolar_medio}")

Total Produzido: 3548338710.0
Total Exportado: 87982432.0
Percentual Exportado: 2.4795387134843168
Dolar Médio: 3.268986666666667


# Pegando dados de consumo, importação, exportação inflação projetada e produção do top 10

In [25]:
df_dados_top10 = pd.read_excel('./top10.xlsx')
df_dados_top10['Ano'] = pd.to_datetime(df_dados_top10['Ano'], format='%Y')
df_dados_top10['Litros'] = df_dados_top10['Litros'].astype(float)
df_dados_top10.sample(10)

,Ano,Pais,Variavel,Litros
250,2008-01-01,Espanha,Produção,"3,591,300,000.00"
521,2011-01-01,Reino Unido,Exportação,"86,600,000.00"
268,2014-01-01,Espanha,Produção,"3,949,400,000.00"
55,2011-01-01,Alemanha,Importação,"1,613,300,000.00"
549,2009-01-01,Uruguai,Exportação,"2,000,000.00"
328,2019-01-01,Reino Unido,Produção,"7,900,000.00"
245,2021-01-01,Paraguai,Importação,"26,000,000.00"
520,2010-01-01,Reino Unido,Exportação,"89,800,000.00"
82,2020-01-01,Alemanha,Importação,"1,448,600,000.00"
529,2019-01-01,Reino Unido,Exportação,"95,400,000.00"


In [53]:
df_pivot = df_dados_top10.pivot_table(index=['Ano', 'Pais'], columns='Variavel', values='Litros', aggfunc='sum').reset_index()
df_pivot.fillna(0, inplace=True)
df_pivot['Saldo'] =  df_pivot['Produção'] + df_pivot['Importação'] - df_pivot['Exportação'] - df_pivot['Consumo']
df_pivot.head()

Variavel,Ano,Pais,Consumo,Exportação,Importação,Produção,Saldo
0,2008-01-01,Alemanha,"2,074,700,000.00","358,000,000.00","1,370,800,000.00","999,100,000.00","-62,800,000.00"
1,2008-01-01,China,"1,404,600,000.00","5,400,000.00","164,900,000.00","1,260,000,000.00","14,900,000.00"
2,2008-01-01,Espanha,"954,200,000.00","1,691,400,000.00","60,700,000.00","3,591,300,000.00","1,006,400,000.00"
3,2008-01-01,Estados Unidos,"2,770,000,000.00","463,800,000.00","825,000,000.00","1,934,000,000.00","-474,800,000.00"
4,2008-01-01,Haiti,"500,000.00",0.00,"500,000.00",0.00,0.00


In [48]:
df_economia_top10 = pd.read_excel('./WEO_Data.xlsx')
df_economia_top10['Ano'] = pd.to_datetime(df_economia_top10['Ano'], format='%Y')
df_pivot_economia = df_economia_top10.pivot_table(index=['Ano', 'Pais'], columns='Variavel', values='Percentual', aggfunc='sum').reset_index()
df_pivot_economia.fillna(0, inplace=True)
df_pivot_economia.head()

Variavel,Ano,Pais,Importação,Inflação
0,2024-01-01,Alemanha,3.39,2.75
1,2024-01-01,China,0.78,1.90
2,2024-01-01,Espanha,4.80,3.37
3,2024-01-01,Estados Unidos,-0.83,2.59
4,2024-01-01,Haiti,1.40,12.68


In [52]:

figs = []

for pais in df_pivot['Pais'].unique():
    df_temp = df_pivot[df_pivot['Pais'] == pais]
    df_temp2 = df_exportacao_top10[df_exportacao_top10['Destino'] == pais]
    
    fig2 = px.bar(df_temp2, 
                  x='Ano', 
                  y='Litros', 
                  title=f'Brasil para {pais} por ano',
                  labels={'Litros': 'Litros Exportados', 'Ano': 'Ano'})
    
    fig2.update_layout(title=f'Brasil para {pais} por ano',
                      xaxis_title='Ano',
                      yaxis_title='Litros',
                      xaxis_tickangle=-45,
                      legend_title_text='',
                      yaxis_type="log",
                      xaxis_title_standoff=30,
                      title_font=dict(size=24), 
                      template="plotly_dark",
                      showlegend=False, 
                      yaxis_showticklabels=False,
                      xaxis=dict(tickvals=df_temp['Ano'], ticktext=df_temp['Ano'].dt.year),
                      annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])
    
    fig2.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)

    fig = px.bar(df_temp,
                 x="Ano",
                 y=["Consumo", "Importação", "Produção", "Exportação"],
                 color_discrete_sequence=["blue", "green", "red","orange"],
                 barmode="group")
    
    fig.update_layout(title=f'Dados - {pais}',
                      xaxis_title='Ano',
                      yaxis_title='Litros',
                      xaxis_tickangle=-45,
                      legend_title_text='',
                      yaxis_type="log",
                      xaxis_title_standoff=30,
                      title_font=dict(size=24), 
                      template="plotly_dark",
                      xaxis=dict(tickvals=df_temp['Ano'], ticktext=df_temp['Ano'].dt.year),
                      annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

    fig1 = go.Figure()
    fig1.add_trace(go.Bar(x=df_temp['Ano'], y=df_temp['Saldo'], name='Saldo'))
    fig1.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)
    fig1.add_trace(go.Scatter(x=df_temp['Ano'], y=df_temp['Saldo'], mode='lines', name='Tendência', line=dict(color='red')))
    
    fig1.update_layout(title=f'Saldo Anual com Linha de Tendência - {pais}',
                    xaxis_title='Ano',
                    yaxis_title='Litros',
                    legend_title_text='',
                    showlegend=False, 
                    title_font=dict(size=24),
                    xaxis_title_standoff=30,
                    template="plotly_dark",
                    xaxis_tickangle=-45,
                    xaxis=dict(tickvals=df_pivot['Ano'], ticktext=df_pivot['Ano'].dt.year),
                    )
    
    
    
    fig3 = go.Figure()
    subset = df_pivot_economia[df_pivot_economia['Pais'] == pais]
    fig3.add_trace(go.Scatter(x=subset['Ano'], y=subset['Inflação'],
                    mode='lines+markers',
                    name=pais))

    fig3.update_layout(title=f'Projeção de Inflação nos Próximos anos - {pais}',
                    xaxis_title='Ano',
                    yaxis_title='Inflação (%)',
                    title_font=dict(size=24),
                    xaxis_title_standoff=30,
                    template="plotly_dark",
                    xaxis=dict(tickvals=subset['Ano'], ticktext=subset['Ano'].dt.year))

    figs.append(fig)
    figs.append(fig1)
    figs.append(fig2)
    figs.append(fig3)

for fig in figs:
    fig.show()


